In [62]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

from xgboost import XGBRegressor

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from tenso


from sklearn.decomposition import PCA

import numpy as np

In [ ]:
def get_esposizione(s=''):

    _l = ['ovest', 'nord', 'sud', 'est']
    _o = {i:0 for i in _l}

    for c in _l:

        if c in s:

            _o[c] = 1
            s = s.replace(c, '')

    return _o 

In [2]:
data = pd.read_excel('output_data_1k.xlsx')

In [3]:
bologna_shp = gpd.read_file('./shapefile/aree-statistiche.geojson')

In [4]:
def get_geo_info(p, shp):

    _ = shp.loc[shp.geometry.apply(lambda x: x.contains(p)), ['codice_area_statistica', 'area_statistica', 'cod_quar', 'quartiere', 'cod_zona', 'zona']]
    
    if _.shape[0] == 0:
        
        _row = {i:'ND' for i in _.columns}
        _ = _.append(_row, ignore_index=True)
        
    return _



In [5]:
data['P'] = data[['longitudine', 'latitudine']].apply(lambda x: Point(x['longitudine'], x['latitudine']), axis=1)

In [6]:
geo_enr = data.P.apply(lambda x: get_geo_info(x,bologna_shp)).to_list()
geo_enr_pdf = pd.concat(geo_enr)
geo_enr_pdf.index = data.index


C:\Users\matte\AppData\Local\Temp\ipykernel_2064\1538752299.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  _ = _.append(_row, ignore_index=True)
C:\Users\matte\AppData\Local\Temp\ipykernel_2064\1538752299.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  _ = _.append(_row, ignore_index=True)
C:\Users\matte\AppData\Local\Temp\ipykernel_2064\1538752299.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  _ = _.append(_row, ignore_index=True)
C:\Users\matte\AppData\Local\Temp\ipykernel_2064\1538752299.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  _ = _.append(_row, ignore_index=True)
C:\Users\matte\AppData\Local\Temp\ipykernel_

In [7]:
data_bak = data.copy()

In [8]:
data = data_bak

In [9]:
data = data.join(geo_enr_pdf)
data = data.loc[~data.euro_mq.isna()]
data = data.loc[(data.prezzo.between(80_000, 2_000_000)) & (data.superficie.between(20, 500)) & (data.euro_mq.between(1_000, 5_000))]
data.set_index('id_ann', inplace=True)

In [10]:
data['round_lat'] = data.latitudine.round(2)
data['round_lng'] = data.longitudine.round(2)

In [11]:
target = ['euro_mq']
features = [i for i in data.columns if i.endswith('_ac_feat')]
features = features + ['ascensore','climatizzato','round_lat','round_lng']
model_data = data.loc[:, ['data_prima_presenza_online', 'superficie', 'prezzo', 'url_ann'] + features + target]

In [12]:
oh_stato = OneHotEncoder(sparse=False)
oh_stato.fit(data[['stato']])
stato_pdf = pd.DataFrame(oh_stato.transform(data[['stato']]), columns=oh_stato.get_feature_names_out(), index=data.index)
stato_pdf.head()

C:\Users\matte\AppData\Roaming\Python\Python38\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,stato_Buono / Abitabile,stato_Da ristrutturare,stato_Non partecipabile,stato_Nuovo / In costruzione,stato_Ottimo / Ristrutturato,stato_Partecipabile,stato_nan
id_ann,,,,,,,
5995,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6002,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6005,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6011,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6013,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
oh_locali = OneHotEncoder(sparse=False)
oh_locali.fit(data[['locali']])
locali_pdf = pd.DataFrame(oh_locali.transform(data[['locali']]), columns=oh_locali.get_feature_names_out(), index=data.index)
locali_pdf.head()

C:\Users\matte\AppData\Roaming\Python\Python38\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,locali_0,locali_1,locali_2,locali_3,locali_4,locali_5,locali_5+,locali_nan
id_ann,,,,,,,,
5995,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6002,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6005,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6011,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6013,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [14]:
oh_bagni = OneHotEncoder(sparse=False)
oh_bagni.fit(data[['bagni_lkp']])
bagni_pdf = pd.DataFrame(oh_bagni.transform(data[['bagni_lkp']]), columns=oh_bagni.get_feature_names_out(), index=data.index)
bagni_pdf.head()

C:\Users\matte\AppData\Roaming\Python\Python38\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,bagni_lkp_##,bagni_lkp_00,bagni_lkp_01,bagni_lkp_02,bagni_lkp_03,bagni_lkp_3+
id_ann,,,,,,
5995,0.0,0.0,0.0,1.0,0.0,0.0
6002,0.0,0.0,1.0,0.0,0.0,0.0
6005,0.0,0.0,1.0,0.0,0.0,0.0
6011,0.0,0.0,1.0,0.0,0.0,0.0
6013,0.0,0.0,0.0,0.0,1.0,0.0


In [15]:
oh_p_totali = OneHotEncoder(sparse=False)
oh_p_totali.fit(data[['piani_totali_lkp']])
p_totali_pdf = pd.DataFrame(oh_p_totali.transform(data[['piani_totali_lkp']]), columns=oh_p_totali.get_feature_names_out(), index=data.index)
p_totali_pdf.head()

C:\Users\matte\AppData\Roaming\Python\Python38\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,piani_totali_lkp_-1,piani_totali_lkp_01,piani_totali_lkp_02,piani_totali_lkp_03,piani_totali_lkp_04,piani_totali_lkp_05,piani_totali_lkp_06,piani_totali_lkp_07,piani_totali_lkp_08,piani_totali_lkp_09,piani_totali_lkp_10,piani_totali_lkp_10+
id_ann,,,,,,,,,,,,
5995,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6005,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6011,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6013,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
oh_piano = OneHotEncoder(sparse=False)
oh_piano.fit(data[['piano_lkp']])
piano_pdf = pd.DataFrame(oh_piano.transform(data[['piano_lkp']]), columns=oh_piano.get_feature_names_out(), index=data.index)
piano_pdf.head()

C:\Users\matte\AppData\Roaming\Python\Python38\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,piano_lkp_ A,piano_lkp_ R,piano_lkp_ S,piano_lkp_ T,piano_lkp_+0,piano_lkp_00,piano_lkp_01,piano_lkp_02,piano_lkp_03,piano_lkp_04,...,piano_lkp_06,piano_lkp_07,piano_lkp_08,piano_lkp_09,piano_lkp_10,piano_lkp_10+,piano_lkp_S2,piano_lkp_S3,piano_lkp_S4,piano_lkp_S5
id_ann,,,,,,,,,,,,,,,,,,,,,
5995,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6002,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6011,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
oh_zona = OneHotEncoder(sparse=False)
oh_zona.fit(data[['zona']])
zona_pdf = pd.DataFrame(oh_zona.transform(data[['zona']]), columns=oh_zona.get_feature_names_out(), index=data.index)
zona_pdf.head()

C:\Users\matte\AppData\Roaming\Python\Python38\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,zona_Barca,zona_Bolognina,zona_Borgo Panigale,zona_Colli,zona_Corticella,zona_Costa Saragozza,zona_Galvani,zona_Irnerio,zona_Lame,zona_Malpighi,zona_Marconi,zona_Mazzini,zona_Murri,zona_ND,zona_S. Ruffillo,zona_S. Viola,zona_S. Vitale,zona_Saffi,zona_San Donato
id_ann,,,,,,,,,,,,,,,,,,,
5995,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6005,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6013,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
oh_anno_c = OneHotEncoder(sparse=False)
oh_anno_c.fit(data[['anno_costruzione_lkp']])
anno_c_pdf = pd.DataFrame(oh_anno_c.transform(data[['anno_costruzione_lkp']]), columns=oh_anno_c.get_feature_names_out(), index=data.index)
anno_c_pdf.head()

C:\Users\matte\AppData\Roaming\Python\Python38\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,anno_costruzione_lkp_0_O_ANNO,anno_costruzione_lkp_1_ANTE_1950,anno_costruzione_lkp_3_1950_1960,anno_costruzione_lkp_4_1960_1970,anno_costruzione_lkp_5_1970,anno_costruzione_lkp_6_1970_1990,anno_costruzione_lkp_7_1990_2010,anno_costruzione_lkp_8_POST_2010
id_ann,,,,,,,,
5995,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6005,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6011,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6013,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
oh_classe_e = OneHotEncoder(sparse=False)
oh_classe_e.fit(data[['classe_energetica_lkp']])
classe_e_pdf = pd.DataFrame(oh_classe_e.transform(data[['classe_energetica_lkp']]), columns=oh_classe_e.get_feature_names_out(), index=data.index)
classe_e_pdf.head()

C:\Users\matte\AppData\Roaming\Python\Python38\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,classe_energetica_lkp_,classe_energetica_lkp_A,classe_energetica_lkp_B,classe_energetica_lkp_C,classe_energetica_lkp_D,classe_energetica_lkp_E,classe_energetica_lkp_F,classe_energetica_lkp_G
id_ann,,,,,,,,
5995,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6002,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6005,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6011,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6013,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
oh_risc_c = OneHotEncoder(sparse=False)
oh_risc_c.fit(data[['riscaldamento_centralizzato_cat']])
risc_c_pdf = pd.DataFrame(oh_risc_c.transform(data[['riscaldamento_centralizzato_cat']]), columns=oh_risc_c.get_feature_names_out(), index=data.index)
risc_c_pdf.head()

C:\Users\matte\AppData\Roaming\Python\Python38\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,riscaldamento_centralizzato_cat_NO,riscaldamento_centralizzato_cat_SI,riscaldamento_centralizzato_cat_nan
id_ann,,,
5995,1.0,0.0,0.0
6002,1.0,0.0,0.0
6005,0.0,1.0,0.0
6011,1.0,0.0,0.0
6013,1.0,0.0,0.0


In [21]:
model_data_feat = model_data[features]
model_data_feat = model_data_feat.join(stato_pdf)
model_data_feat = model_data_feat.join(locali_pdf)
model_data_feat = model_data_feat.join(bagni_pdf)
model_data_feat = model_data_feat.join(p_totali_pdf)
model_data_feat = model_data_feat.join(piano_pdf)
model_data_feat = model_data_feat.join(zona_pdf)
model_data_feat = model_data_feat.join(anno_c_pdf)
model_data_feat = model_data_feat.join(classe_e_pdf)
model_data_feat = model_data_feat.join(risc_c_pdf)

model_data_feat = model_data_feat.drop('_ac_feat', axis=1)

model_data_feat.head()

,balcone_ac_feat,cantina_ac_feat,arredato_ac_feat,giardino privato_ac_feat,terrazza_ac_feat,fibra ottica_ac_feat,impianto tv centralizzato_ac_feat,giardino comune_ac_feat,infissi esterni in doppio vetro / pvc_ac_feat,esposizione doppia_ac_feat,...,classe_energetica_lkp_A,classe_energetica_lkp_B,classe_energetica_lkp_C,classe_energetica_lkp_D,classe_energetica_lkp_E,classe_energetica_lkp_F,classe_energetica_lkp_G,riscaldamento_centralizzato_cat_NO,riscaldamento_centralizzato_cat_SI,riscaldamento_centralizzato_cat_nan
id_ann,,,,,,,,,,,,,,,,,,,,,
5995,1,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
6002,1,0,1,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
6005,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
6011,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6013,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [22]:
f = ['1 balcone_ac_feat','2 balconi_ac_feat','3 balconi_ac_feat','4 balconi_ac_feat']
balconi_n = pd.DataFrame(model_data_feat[f].sum(axis=1), columns=['n'])
balconi_idmax = pd.DataFrame(model_data_feat[f].idxmax(axis=1), columns=['idmax'])
balconi = balconi_n.join(balconi_idmax)
balconi['numero_balconi'] = balconi.apply(lambda x: 0 if x['n'] == 0 else int(x['idmax'][0]), axis=1)

In [23]:
model_data_feat = model_data_feat.join(balconi[['numero_balconi']]).drop(f, axis=1)

In [48]:
f = ['esposizione est, ovest_ac_feat', 
     'esposizione est_ac_feat', 
     'esposizione nord, est, ovest_ac_feat',
     'esposizione nord, est_ac_feat',
     'esposizione nord, ovest_ac_feat',
     'esposizione nord, sud, est, ovest_ac_feat',	
     'esposizione nord, sud, est_ac_feat',
     'esposizione nord, sud, ovest_ac_feat',
     'esposizione nord, sud_ac_feat', 
     'esposizione nord_ac_feat', 
     'esposizione ovest_ac_feat',
     'esposizione sud, est, ovest_ac_feat',
     'esposizione sud, est_ac_feat',
     'esposizione sud, ovest_ac_feat',
     'esposizione sud_ac_feat'
]

esposizione_n = pd.DataFrame(model_data_feat[f].sum(axis=1), columns=['n'])
esposizione_idmax = pd.DataFrame(model_data_feat[f].idxmax(axis=1), columns=['idmax'])
esposizione = esposizione_n.join(esposizione_idmax)
esposizione = esposizione.apply(lambda x: get_esposizione() if x['n'] == 0 else get_esposizione(x['idmax']), axis=1)
esposizione = pd.DataFrame(esposizione.to_list(), index=esposizione.index)
esposizione.columns = ['esposizione_'+i for i in esposizione.columns]

In [50]:
model_data_feat = model_data_feat.join(esposizione).drop(f, axis=1)

In [51]:
(model_data.data_prima_presenza_online >= '2023-06-20').value_counts()

False    23913
True       667
Name: data_prima_presenza_online, dtype: int64

In [111]:
training_data = model_data.loc[model_data.data_prima_presenza_online < '2023-06-20', ['euro_mq', 'superficie']]
#training_data.join(model_data_feat, how='inner').to_excel('training_w_feat.xlsx')
training_y = training_data[['euro_mq']]
training_X = training_data.join(model_data_feat, how='inner')

train_dict = {
    'lth_50':
        {   
            'raw':training_X.loc[training_X.superficie.between(  0, 50)],
            'X':training_X.loc[training_X.superficie.between(  0, 50)].drop(['euro_mq', 'superficie', 'round_lat', 'round_lng'], axis=1).copy(),
            'y':training_y.join(training_X.loc[training_X.superficie.between(  0, 50)], how='inner', rsuffix='_X')[['euro_mq']]
        
        },

    '50_100':
        {
            'raw':training_X.loc[training_X.superficie.between( 50, 100)],
            'X':training_X.loc[training_X.superficie.between( 50, 100)].drop(['euro_mq', 'superficie', 'round_lat', 'round_lng'], axis=1).copy(),
            'y':training_y.join(training_X.loc[training_X.superficie.between( 50, 100)], how='inner', rsuffix='_X')[['euro_mq']]
        
        },
    'l00_130':
        {
            'raw':training_X.loc[training_X.superficie.between(100, 130)],
            'X':training_X.loc[training_X.superficie.between(100, 130)].drop(['euro_mq', 'superficie', 'round_lat', 'round_lng'], axis=1).copy(),
            'y':training_y.join(training_X.loc[training_X.superficie.between(100, 130)], how='inner', rsuffix='_X')[['euro_mq']]
            
        },
    'gth_130':
        {
            'raw':training_X.loc[training_X.superficie > 130],
            'X':training_X.loc[training_X.superficie > 130].drop(['euro_mq', 'superficie', 'round_lat', 'round_lng'], axis=1).copy(),
            'y':training_y.join(training_X.loc[training_X.superficie > 130], how='inner', rsuffix='_X')[['euro_mq']]
        }
}

training_X = training_X.drop(['euro_mq', 'superficie', 'round_lat', 'round_lng'], axis=1)

print('TRAIN DICT')
for k, v in train_dict.items():
    print(f'SET {k}: X [{v["X"].shape}] - y [{v["y"].shape}] - RAW [{v["raw"].shape}]')

test_data = model_data.loc[model_data.data_prima_presenza_online >= '2023-06-20', ['euro_mq', 'superficie']]
test_y = test_data[['euro_mq']]
test_X = test_data.join(model_data_feat, how='inner')#.drop(['euro_mq', 'superficie', 'round_lat', 'round_lng'], axis=1)

test_dict = {
    'lth_50':
        {
            'raw':test_X.loc[test_X.superficie.between(  0, 50)],
            'X':test_X.loc[test_X.superficie.between(  0, 50)].drop(['euro_mq', 'superficie', 'round_lat', 'round_lng'], axis=1).copy(),
            'y':test_y.join(test_X.loc[test_X.superficie.between(  0, 50)], how='inner', rsuffix='_X')[['euro_mq']]
        
        },

    '50_100':
        {
            'raw':test_X.loc[test_X.superficie.between( 50, 100)],
            'X':test_X.loc[test_X.superficie.between( 50, 100)].drop(['euro_mq', 'superficie', 'round_lat', 'round_lng'], axis=1).copy(),
            'y':test_y.join(test_X.loc[test_X.superficie.between( 50, 100)], how='inner', rsuffix='_X')[['euro_mq']]
        
        },
    'l00_130':
        {
            'raw':test_X.loc[test_X.superficie.between(100, 130)],
            'X':test_X.loc[test_X.superficie.between(100, 130)].drop(['euro_mq', 'superficie', 'round_lat', 'round_lng'], axis=1).copy(),
            'y':test_y.join(test_X.loc[test_X.superficie.between(100, 130)], how='inner', rsuffix='_X')[['euro_mq']]
            
        },
    'gth_130':
        {
            'raw':test_X.loc[test_X.superficie > 130],
            'X':test_X.loc[test_X.superficie > 130].drop(['euro_mq', 'superficie', 'round_lat', 'round_lng'], axis=1).copy(),
            'y':test_y.join(test_X.loc[test_X.superficie > 130], how='inner', rsuffix='_X')[['euro_mq']]
        }
}

test_X = test_X.drop(['euro_mq', 'superficie', 'round_lat', 'round_lng'], axis=1)

print('TEST DICT')
for k, v in test_dict.items():
    print(f'SET {k}: X [{v["X"].shape}] - y [{v["y"].shape}] - RAW [{v["raw"].shape}] ')


TRAIN DICT
SET lth_50: X [(2983, 139)] - y [(2983, 1)] - RAW [(2983, 143)]
SET 50_100: X [(12970, 139)] - y [(12970, 1)] - RAW [(12970, 143)]
SET l00_130: X [(5709, 139)] - y [(5709, 1)] - RAW [(5709, 143)]
SET gth_130: X [(3557, 139)] - y [(3557, 1)] - RAW [(3557, 143)]
TEST DICT
SET lth_50: X [(64, 139)] - y [(64, 1)] - RAW [(64, 143)] 
SET 50_100: X [(385, 139)] - y [(385, 1)] - RAW [(385, 143)] 
SET l00_130: X [(146, 139)] - y [(146, 1)] - RAW [(146, 143)] 
SET gth_130: X [(107, 139)] - y [(107, 1)] - RAW [(107, 143)] 


In [86]:
(training_X.sum() / training_X.shape[0]).sort_values()

canna fumaria_ac_feat                 0.000042
piano_lkp_S5                          0.000042
piano_lkp_S3                          0.000042
piano_lkp_S4                          0.000042
piano_lkp_ A                          0.000084
                                        ...   
impianto tv centralizzato_ac_feat     0.520261
riscaldamento_centralizzato_cat_NO    0.523857
porta blindata_ac_feat                0.565216
bagni_lkp_01                          0.646928
cantina_ac_feat                       0.673734
Length: 139, dtype: float64

In [87]:
pca = PCA(40)
pca.fit(training_X)

PCA(n_components=40)

In [55]:
pca.explained_variance_ratio_.cumsum()

array([0.07598113, 0.13082498, 0.18252144, 0.22267831, 0.25444277,
       0.28313275, 0.3082202 , 0.33195149, 0.35357017, 0.3745248 ,
       0.39500819, 0.41441486, 0.43328403, 0.45152839, 0.46941042,
       0.48679221, 0.50375306, 0.51999884, 0.53592704, 0.55115776,
       0.56592856, 0.58031161, 0.5934352 , 0.60632316, 0.61837135,
       0.63027748, 0.64177747, 0.65302907, 0.6640031 , 0.67444098,
       0.68478082, 0.69456772, 0.70413621, 0.71319711, 0.72197164,
       0.73068187, 0.73921776, 0.74755073, 0.75534483, 0.76279605])

In [118]:
lr = LinearRegression(n_jobs=-1, fit_intercept=True)
lr.fit(training_X, np.log(training_y))
train_score = lr.score(training_X, np.log(training_y))
test_score = lr.score(test_X, np.log(test_y))

print(f'LR - TRAIN SCORE: {train_score:.2f} - TEST SCORE: {test_score:.2f}')


LR - TRAIN SCORE: 0.53 - TEST SCORE: 0.49


In [70]:
training_data['predicted_eur_mq'] = np.exp(lr.predict(training_X))
training_data['predicted_price'] = training_data['predicted_eur_mq'] * training_data['superficie']

mse_train = np.sqrt(np.square(training_data['euro_mq']-training_data['predicted_eur_mq']).mean())

test_data['predicted_eur_mq'] = np.exp(lr.predict(test_X))
test_data['predicted_price'] = test_data['predicted_eur_mq'] * test_data['superficie']

mse_test = np.sqrt(np.square(test_data['euro_mq']-test_data['predicted_eur_mq']).mean())

print(f'LR - MSE TRAIN: {mse_train:.2f} - MSE TEST: {mse_test:.2f}')


LR - MSE TRAIN: 549.11 - MSE TEST: inf


C:\Users\matte\AppData\Local\Temp\ipykernel_2064\2838955060.py:6: RuntimeWarning: overflow encountered in exp
  test_data['predicted_eur_mq'] = np.exp(lr.predict(test_X))


In [119]:
lr_model_dict = {}

for k, v in train_dict.items():

    lr_model_dict[k] = LinearRegression(n_jobs=-1, fit_intercept=True).fit(v['X'], np.log(v['y']))


for k, lr_mod in lr_model_dict.items():

    train_score = lr_mod.score(train_dict[k]['X'], np.log(train_dict[k]['y']))
    test_score = lr_mod.score(test_dict[k]['X'], np.log(test_dict[k]['y']))
    print(f"""{k}: TRAIN {train_score:.2f} - TEST {test_score:.2f}""")


lth_50: TRAIN 0.43 - TEST -7368015738694029132431360.00
50_100: TRAIN 0.56 - TEST -31857733990735402762240.00
l00_130: TRAIN 0.57 - TEST 0.55
gth_130: TRAIN 0.55 - TEST 0.34


In [121]:
for k, lr_mod in lr_model_dict.items():

    train_dict[k]['raw']['predicted_eur_mq'] = np.exp(lr_mod.predict(train_dict[k]['X']))
    train_dict[k]['raw']['predicted_price'] = train_dict[k]['raw']['predicted_eur_mq'] * train_dict[k]['raw']['superficie']

    mse_train = np.sqrt(np.square(train_dict[k]['raw']['euro_mq']-train_dict[k]['raw']['predicted_eur_mq']).mean())

    test_dict[k]['raw']['predicted_eur_mq'] = np.exp(lr_mod.predict(test_dict[k]['X']))
    test_dict[k]['raw']['predicted_price'] = test_dict[k]['raw']['predicted_eur_mq'] * test_dict[k]['raw']['superficie']

    mse_test = np.sqrt(np.square(test_dict[k]['raw']['euro_mq']-test_dict[k]['raw']['predicted_eur_mq']).mean())

    print(f'[{k}] - LR - MSE TRAIN: {mse_train:.2f} - MSE TEST: {mse_test:.2f}')

train_raw = pd.concat([v['raw'] for k, v in train_dict.items()])
test_raw = pd.concat([v['raw'] for k, v in test_dict.items()])

mse_train = np.sqrt(np.square(train_raw['euro_mq']-train_raw['predicted_eur_mq']).mean())
mse_test = np.sqrt(np.square(test_raw['euro_mq']-test_raw['predicted_eur_mq']).mean())

print(f'LR - MSE TRAIN: {mse_train:.2f} - MSE TEST: {mse_test:.2f}')

[lth_50] - LR - MSE TRAIN: 481.03 - MSE TEST: 849.31
[50_100] - LR - MSE TRAIN: 516.47 - MSE TEST: inf
[l00_130] - LR - MSE TRAIN: 489.90 - MSE TEST: 526.29
[gth_130] - LR - MSE TRAIN: 569.18 - MSE TEST: 632.18
LR - MSE TRAIN: 514.37 - MSE TEST: inf


C:\Users\matte\AppData\Local\Temp\ipykernel_2064\353620736.py:8: RuntimeWarning: overflow encountered in exp
  test_dict[k]['raw']['predicted_eur_mq'] = np.exp(lr_mod.predict(test_dict[k]['X']))


In [81]:
rf = RandomForestRegressor(n_jobs=-1,# n_estimators=50, min_samples_leaf=10,
                           criterion='squared_error')

rf.fit(training_X, training_y)

train_score = rf.score(training_X, training_y)
test_score = rf.score(test_X, test_y)

print(f'RF - TRAIN SCORE: {train_score:.2f} - TEST SCORE: {test_score:.2f}')

C:\Users\matte\AppData\Local\Temp\ipykernel_2064\2570628723.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(training_X, training_y)


RF - TRAIN SCORE: 0.95 - TEST SCORE: 0.66


In [82]:
training_data['predicted_eur_mq'] = rf.predict(training_X)
training_data['predicted_price'] = training_data['predicted_eur_mq'] * training_data['superficie']

mse_train = np.sqrt(np.square(training_data['euro_mq']-training_data['predicted_eur_mq']).mean())

test_data['predicted_eur_mq'] = rf.predict(test_X)
test_data['predicted_price'] = test_data['predicted_eur_mq'] * test_data['superficie']

mse_test = np.sqrt(np.square(test_data['euro_mq']-test_data['predicted_eur_mq']).mean())

print(f'RF - MSE TRAIN: {mse_train:.2f} - MSE TEST: {mse_test:.2f}')


RF - MSE TRAIN: 173.77 - MSE TEST: 457.99


In [109]:
xgb = XGBRegressor(n_jobs=-1, grow_policy='lossguide')
xgb.fit(training_X, np.log(training_y))

train_score = xgb.score(training_X, np.log(training_y))
test_score = xgb.score(test_X, np.log(test_y))

print(f'XGB - TRAIN SCORE: {train_score:.2f} - TEST SCORE: {test_score:.2f}')

XGB - TRAIN SCORE: 0.77 - TEST SCORE: 0.59


In [110]:
training_data['predicted_eur_mq'] = np.exp(xgb.predict(training_X))
training_data['predicted_price'] = training_data['predicted_eur_mq'] * training_data['superficie']

mse_train = np.sqrt(np.square(training_data['euro_mq']-training_data['predicted_eur_mq']).mean())

test_data['predicted_eur_mq'] = np.exp(xgb.predict(test_X))
test_data['predicted_price'] = test_data['predicted_eur_mq'] * test_data['superficie']

mse_test = np.sqrt(np.square(test_data['euro_mq']-test_data['predicted_eur_mq']).mean())

print(f'XGB - MSE TRAIN: {mse_train:.2f} - MSE TEST: {mse_test:.2f}')

XGB - MSE TRAIN: 382.68 - MSE TEST: 475.20


In [116]:
xgb_model_dict = {}

for k, v in train_dict.items():

    xgb_model_dict[k] = XGBRegressor(n_jobs=-1, grow_policy='lossguide').fit(v['X'], np.log(v['y']))


for k, xgb_m in xgb_model_dict.items():

    train_score = xgb_m.score(train_dict[k]['X'], np.log(train_dict[k]['y']))
    test_score = xgb_m.score(test_dict[k]['X'], np.log(test_dict[k]['y']))
    print(f"""{k}: TRAIN {train_score:.2f} - TEST {test_score:.2f}""")


lth_50: TRAIN 0.89 - TEST 0.42
50_100: TRAIN 0.82 - TEST 0.60
l00_130: TRAIN 0.90 - TEST 0.55
gth_130: TRAIN 0.92 - TEST 0.52


In [117]:
for k, xgb_m in xgb_model_dict.items():

    train_dict[k]['raw']['predicted_eur_mq'] = np.exp(xgb_m.predict(train_dict[k]['X']))
    train_dict[k]['raw']['predicted_price'] = train_dict[k]['raw']['predicted_eur_mq'] * train_dict[k]['raw']['superficie']

    mse_train = np.sqrt(np.square(train_dict[k]['raw']['euro_mq']-train_dict[k]['raw']['predicted_eur_mq']).mean())

    test_dict[k]['raw']['predicted_eur_mq'] = np.exp(xgb_m.predict(test_dict[k]['X']))
    test_dict[k]['raw']['predicted_price'] = test_dict[k]['raw']['predicted_eur_mq'] * test_dict[k]['raw']['superficie']

    mse_test = np.sqrt(np.square(test_dict[k]['raw']['euro_mq']-test_dict[k]['raw']['predicted_eur_mq']).mean())

    print(f'[{k}] - XGB - MSE TRAIN: {mse_train:.2f} - MSE TEST: {mse_test:.2f}')

train_raw = pd.concat([v['raw'] for k, v in train_dict.items()])
test_raw = pd.concat([v['raw'] for k, v in test_dict.items()])

mse_train = np.sqrt(np.square(train_raw['euro_mq']-train_raw['predicted_eur_mq']).mean())
mse_test = np.sqrt(np.square(test_raw['euro_mq']-test_raw['predicted_eur_mq']).mean())

print(f'XGB - MSE TRAIN: {mse_train:.2f} - MSE TEST: {mse_test:.2f}')

[lth_50] - XGB - MSE TRAIN: 215.13 - MSE TEST: 458.83
[50_100] - XGB - MSE TRAIN: 330.45 - MSE TEST: 450.52
[l00_130] - XGB - MSE TRAIN: 238.18 - MSE TEST: 570.31
[gth_130] - XGB - MSE TRAIN: 244.04 - MSE TEST: 517.68
XGB - MSE TRAIN: 287.88 - MSE TEST: 488.87


In [149]:
data.join(test_data[['predicted_eur_mq', 'predicted_price']], how='inner').to_excel('test_data_zona_pca.xlsx')